In [24]:
import json
import requests
from pprint import pprint

In [25]:
print("Based on the confirmed details, here is the Master Scenario Events List: \n\n1. 00:00 - Initial Alert - Inject - Control Room Operator to On Scene Commander - \"H2S gas detectors in Unit OD1 have triggered an alarm indicating elevated H2S levels.\" - On Scene Commander initiates initial assessment and alerts Emergency Response Team (ERT). - Tests early detection and initial notification procedures under OPITO MEMIR. - Ensure rapid recognition and communication of hazardous gas detection.\n\n2. 00:05 - Situation Confirmation - Inject - On Scene Commander to HSE Officer and ERT Leader - \"Confirm H2S concentration levels and assess affected areas.\" - HSE Officer verifies gas levels; ERT Leader prepares personnel with breathing apparatus. - Tests hazard assessment and PPE deployment readiness.\n\n3. 00:10 - Emergency Declaration - Inject - On Scene Commander to Company HQ Observer and all platform personnel - \"Declare H2S gas release emergency; initiate emergency shutdown procedures and evacuation protocols.\" - Control Room Operator initiates emergency shutdown; Deck Supervisor coordinates personnel evacuation to muster points. - Tests emergency declaration, shutdown, and evacuation coordination.\n\n4. 00:15 - External Notification - Inject - On Scene Commander to Coast Guard and Environmental Regulator - \"Notify external agencies of H2S release and potential environmental impact.\" - Coast Guard prepares for possible offshore assistance; Environmental Regulator requests monitoring data. - Tests external communication and regulatory notification.\n\n5. 00:20 - Environmental Concern Raised - Inject - Fisherman Representative to On Scene Commander - \"Concern expressed about potential impact on nearby fishing zones.\" - On Scene Commander acknowledges and coordinates with Environmental Regulator for impact assessment. - Tests stakeholder communication and environmental risk management.\n\n6. 00:25 - Medical Alert - Inject - Medical Officer to On Scene Commander - \"Report of personnel exhibiting symptoms consistent with H2S exposure.\" - Medical Officer initiates medical triage and requests emergency medical evacuation if needed. - Tests medical response and casualty management.\n\n7. 00:30 - Spill Containment - Inject - ERT Leader to Logistics Coordinator - \"Report of minor hydrocarbon oil spill near containment area.\" - Logistics Coordinator deploys spill kits and containment measures. - Tests spill response and resource coordination.\n\n8. 00:40 - Communication System Check - Inject - Control Room Operator to all teams - \"Verify functionality of all communication systems including VHF radios and satellite phones.\" - Teams confirm operational status; report any failures. - Tests communication reliability under emergency conditions.\n\n9. 00:50 - Emergency Shutdown Verification - Inject - Control Room Operator to On Scene Commander - \"Confirm all emergency shutdown systems are fully engaged and stable.\" - On Scene Commander verifies system status and reports to Company HQ Observer. - Tests shutdown system effectiveness and reporting.\n\n10. 01:00 - Situation Update and Demobilization Planning - Inject - On Scene Commander to all response teams and Company HQ Observer - \"Provide status update; begin planning phased demobilization as gas levels return to safe limits.\" - Teams prepare for safe return to normal operations following OPITO MEMIR guidelines. - Tests incident resolution, recovery planning, and debriefing procedures.")

Based on the confirmed details, here is the Master Scenario Events List: 

1. 00:00 - Initial Alert - Inject - Control Room Operator to On Scene Commander - "H2S gas detectors in Unit OD1 have triggered an alarm indicating elevated H2S levels." - On Scene Commander initiates initial assessment and alerts Emergency Response Team (ERT). - Tests early detection and initial notification procedures under OPITO MEMIR. - Ensure rapid recognition and communication of hazardous gas detection.

2. 00:05 - Situation Confirmation - Inject - On Scene Commander to HSE Officer and ERT Leader - "Confirm H2S concentration levels and assess affected areas." - HSE Officer verifies gas levels; ERT Leader prepares personnel with breathing apparatus. - Tests hazard assessment and PPE deployment readiness.

3. 00:10 - Emergency Declaration - Inject - On Scene Commander to Company HQ Observer and all platform personnel - "Declare H2S gas release emergency; initiate emergency shutdown procedures and evacuation

In [26]:
def normalize_trainee_roles(raw_roles: list[dict]) -> list[dict]:
    """
    Normalize trainee_roles into object-style access control format.
    """
    normalized = []

    for role in raw_roles:
        access = role.get("accessControl", {})

        normalized.append({
            "role": role.get("role", "Unknown"),
            "accessControl": {
                "phone": {
                    "access": access.get("phoneAccess") or "none"
                },
                "radio": {
                    "access": access.get("radioAccess") or "none",
                    "channels": access.get("channels", [])
                },
                "board": {
                    "access": access.get("board", {}).get("access", "none"),
                    "description": access.get("board", {}).get("description")
                }
            }
        })

    return normalized


In [35]:
def extract_required_data(data: dict) -> dict:
    """
    Extract required data from the input dictionary.
    """
    REQUIRED_FIELDS = {
        "unit_name": None,
        "unit_type": None,
        "asset_name": None,
        "asset_type": None,
        "asset_location": None,
        "ownership_operator_name": None,
        "workforce_size_shift": None,
        "primary_function": None,
        "key_processes": None,
        "hazardous_materials": None,
        "response_equipment": None,
        "communication_systems": None,
        "environmental_conditions": None,
        "proximity_sensitive_areas": None,
        "trainee_roles": None,
    }

    collected = data.get("collectedUnitInfo", {})
    # print(collected)
    # Check if *all* values are None
    # all_null = all(v is None for v in collected.values())

    if collected != None:
        # Use collectedUnitInfo if available
        REQUIRED_FIELDS.update({
            "unit_name": collected.get("unitName"),
            "unit_type": collected.get("unitType"),
            "asset_name": collected.get("assetName"),
            "asset_type": collected.get("assetType"),
            "asset_location": collected.get("assetLocation"),
            "ownership_operator_name": collected.get("ownershipOperatorName"),
            "workforce_size_shift": collected.get("workforceSizeShift"),
            "primary_function": collected.get("primaryFunction"),
            "key_processes": collected.get("keyProcesses"),
            "hazardous_materials": collected.get("hazardousMaterials"),
            "response_equipment": collected.get("responseEquipment"),
            "communication_systems": collected.get("communicationSystems"),
            "environmental_conditions": collected.get("environmentalConditions"),
            "proximity_sensitive_areas": collected.get("proximitySensitiveAreas"),
        })
    else:
        # Fallback to unitInfo
        unit_info = data.get("unitInfo", {})
        asset = unit_info.get("asset", {})
        org = asset.get("organisation", {})

        REQUIRED_FIELDS.update({
            "unit_name": unit_info.get("name"),
            "unit_type": unit_info.get("type"),
            "asset_name": asset.get("name"),
            "asset_type": asset.get("type"),
            "asset_location": asset.get("country"),
            "ownership_operator_name": org.get("name"),
        })

    # # Extract trainee roles from teamsAssigned
    # teams = data.get("teamsAssigned", [])
    # trainee_roles = []
    # for team in teams:
    #     for user in team.get("users", []):
    #         role = user.get("roleName")
    #         if role:
    #             trainee_roles.append(role)

    # REQUIRED_FIELDS["trainee_roles"] = ", ".join(trainee_roles) if trainee_roles else None

    # Extract trainee roles with flexible access control
    teams = data.get("teamsAssigned", [])
    trainee_roles = []

    for team in teams:
        for user in team.get("users", []):
            role = user.get("roleName")
            access = user.get("accessControl", {})

            if role:
                # Start with a shallow copy of accessControl so we don’t lose new keys
                access_info = dict(access)

                # Normalise the `board` if it exists and is a dict
                if "board" in access and isinstance(access["board"], dict):
                    access_info["board"] = {
                        "access": access["board"].get("access"),
                        "title": access["board"].get("data", {}).get("title"),
                        "description": access["board"].get("data", {}).get("description")
                    }
                elif "board" in access:
                    access_info["board"] = None  # board is explicitly null

                role_info = {
                    "role": role,
                    "accessControl": access_info
                }
                trainee_roles.append(role_info)

    REQUIRED_FIELDS["trainee_roles"] = trainee_roles if trainee_roles else None
    # REQUIRED_FIELDS["trainee_roles"] = normalize_trainee_roles(trainee_roles) if trainee_roles else None

    # Return as JSON string
    return json.dumps(REQUIRED_FIELDS, indent=2)

In [28]:
UNIT_API_BASE = "http://134.209.148.133:3001/api/ai-scenario-generation/hierarchy/unit"

In [33]:
resp = requests.get(f"{UNIT_API_BASE}/{'11'}")
pprint(resp.json())
print(type(resp.json()))

{'collectedUnitInfo': {'assetLocation': 'Houston, Texas, USA',
                       'assetName': 'ChemCo Facility',
                       'assetType': 'Industrial Chemical Plant',
                       'communicationSystems': 'Two-way radios, landline '
                                               'phones, and emergency alert '
                                               'systems',
                       'environmentalConditions': 'Subtropical climate with '
                                                  'occasional hurricanes',
                       'hazardousMaterials': 'Various flammable and toxic '
                                             'chemicals',
                       'keyProcesses': 'Chemical synthesis, storage, '
                                       'packaging, and distribution',
                       'ownershipOperatorName': 'ChemCo Corporation',
                       'primaryFunction': 'Production of industrial chemicals '
                            

In [36]:
print(extract_required_data(resp.json()))

{
  "unit_name": "Central Industrial Complex",
  "unit_type": "Chemical Manufacturing Plant",
  "asset_name": "ChemCo Facility",
  "asset_type": "Industrial Chemical Plant",
  "asset_location": "Houston, Texas, USA",
  "ownership_operator_name": "ChemCo Corporation",
  "workforce_size_shift": "300 employees working in two 12-hour shifts",
  "primary_function": "Production of industrial chemicals and solvents",
  "key_processes": "Chemical synthesis, storage, packaging, and distribution",
  "hazardous_materials": "Various flammable and toxic chemicals",
  "response_equipment": "Fire suppression systems, spill containment kits, and emergency medical supplies",
  "communication_systems": "Two-way radios, landline phones, and emergency alert systems",
  "environmental_conditions": "Subtropical climate with occasional hurricanes",
  "proximity_sensitive_areas": "Adjacent to residential neighborhoods and a river",
  "trainee_roles": [
    {
      "role": "Radio Operator",
      "accessContro